In [109]:
import requests
from requests.exceptions import HTTPError

import pandas as pd
from decimal import Decimal
from pandas.io.json import json_normalize
import json

import numpy as np
import datetime as dt
from pandas_datareader import data as pdr



#Variables/Params
api_key = '46583da07984f6f5e5acc0442fad24656e164280da7811e9aefa8bcfc9d1' #defi pulse


In [110]:
#get json query
def defipulse_getdata(request, api_key):
    try:
        response = requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/' + request +'?api-key=' + api_key + '&period=1y')
        response.raise_for_status()
        # access JSOn content
        return response.json()
        #print("Entire JSON response")
        #print(api_output[0]) #validate response from API

    except HTTPError as http_err:
        return (f'HTTP error occurred: {http_err}')
    except Exception as err:
        return (f'Other error occurred: {err}')

#convert to callable dataframe
def get_lendrates(data):
    df = pd.DataFrame.from_dict(data)
    timestamps = df['timestamp']
    timestamps = pd.to_datetime(timestamps,unit='s').dt.date
    data = pd.json_normalize(df['lend_rates'])
    return pd.merge(timestamps, data, left_index=True, right_index=True)



In [111]:
api_lending_data = defipulse_getdata('getLendingHistory', api_key) #lending history
api_tvl =  defipulse_getdata('GetHistory', api_key) 


In [112]:
api_lending_data_db = get_lendrates(api_lending_data)
api_lending_data_db.melt(id_vars=['timestamp'], ignore_index=True)




,timestamp,variable,value
0,2021-08-12,maker,0.010007
1,2021-08-11,maker,0.010007
2,2021-08-10,maker,0.010007
3,2021-08-09,maker,0.010007
4,2021-08-08,maker,0.010007
...,...,...,...
2185,2020-08-17,definer,NaN
2186,2020-08-16,definer,NaN
2187,2020-08-15,definer,NaN
2188,2020-08-14,definer,NaN


In [141]:
def get_tvl(data):
    df = pd.DataFrame.from_dict(data)
    df.timestamp = pd.to_datetime(df.timeStamp)
    return pd.merge(timestamps, data, left_index=True, right_index=True)

api_tvl
makerlend = get_tvl(api_tvl)
makerlend


AttributeError: 'DataFrame' object has no attribute 'timeStamp'